In [ ]:
from bayes_opt import BayesianOptimization
import numpy as np
from  matplotlib import pyplot as plt
from bayes_opt import UtilityFunction
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs
from scipy.optimize import NonlinearConstraint
pbounds = {'x': (2, 4), 'y': (-3, 3)}
def target_function(x, y):
    # Gardner is looking for the minimum, but this packages looks for maxima, thus the sign switch
    return np.cos(2*x)*np.cos(y) + np.sin(x)

def constraint_function_2_dim(x, y):
    return np.array([
        - np.cos(x) * np.cos(y) + np.sin(x) * np.sin(y),
        - np.cos(x) * np.cos(-y) + np.sin(x) * np.sin(-y)])

constraint_lower = np.array([-np.inf, -np.inf])
constraint_upper = np.array([0.6, 0.6])

constraint = NonlinearConstraint(constraint_function_2_dim, constraint_lower, constraint_upper)
optimizer = BayesianOptimization(
    f=target_function,
    constraint=constraint,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

logger = JSONLogger(path="./logs.log")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)
acquision=UtilityFunction('ucb',kappa=2.5, xi=0.0)
point=optimizer.suggest(acquision)
y=target_function(point['x'],point['y'])
c=constraint_function_2_dim(point['x'],point['y'])
optimizer.register(params=point,target=y,constraint_value=c)